# Multiple tools with OpenAI
## Introduction
An example to have OpenAI use multiple tools at the same time

## Installation

In [1]:
%pip install -q openai


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Setting up the client

In [2]:
from openai import OpenAI
import json

client = OpenAI()

## Setting up the functions

In [3]:
todo_list = []

def get_current_weather(location, unit="celsius"):
    """Get the current weather in a given location"""
    # This is a mock function. In a real scenario, you'd call a weather API here.
    weather_info = {
        "location": location,
        "temperature": 22,
        "unit": unit,
        "forecast": ["sunny", "windy"]
    }
    return json.dumps(weather_info)

def convert_currency(amount, from_currency, to_currency):
    """Convert an amount from one currency to another"""
    # This is a mock function. In a real scenario, you'd call a currency API here.
    # Using fixed rates for demonstration
    rates = {"USD": 1, "EUR": 0.85, "GBP": 0.73, "JPY": 110.14}
    if from_currency not in rates or to_currency not in rates:
        return json.dumps({"error": "Unsupported currency"})
    converted = amount / rates[from_currency] * rates[to_currency]
    return json.dumps({
        "amount": round(converted, 2),
        "from": from_currency,
        "to": to_currency
    })

def manage_todo(action, item="", id=None):
    """Manage a todo list"""
    global todo_list
    if action == "add":
        todo_list.append({"id": len(todo_list) + 1, "item": item, "done": False})
        return json.dumps({"status": "added", "item": item})
    elif action == "list":
        return json.dumps(todo_list)
    elif action == "complete":
        for todo in todo_list:
            if todo["id"] == id:
                todo["done"] = True
                return json.dumps({"status": "completed", "item": todo["item"]})
        return json.dumps({"error": "Item not found"})
    return json.dumps({"error": "Invalid action"})

def calculate(operation, x, y):
    """Perform a simple calculation"""
    operations = {
        "add": lambda a, b: a + b,
        "subtract": lambda a, b: a - b,
        "multiply": lambda a, b: a * b,
        "divide": lambda a, b: a / b if b != 0 else "Error: Division by zero"
    }
    if operation not in operations:
        return json.dumps({"error": "Unsupported operation"})
    result = operations[operation](x, y)
    return json.dumps({
        "operation": operation,
        "x": x,
        "y": y,
        "result": result
    })


## Selecting the right function

In [4]:
def run_conversation(user_input):
    messages = [{"role": "user", "content": user_input}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "The city and state, e.g. San Francisco, CA"},
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
                    },
                    "required": ["location"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "convert_currency",
                "description": "Convert an amount from one currency to another",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "amount": {"type": "number", "description": "The amount to convert"},
                        "from_currency": {"type": "string", "description": "The currency to convert from"},
                        "to_currency": {"type": "string", "description": "The currency to convert to"}
                    },
                    "required": ["amount", "from_currency", "to_currency"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "manage_todo",
                "description": "Manage a todo list",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "action": {"type": "string", "enum": ["add", "list", "complete"]},
                        "item": {"type": "string", "description": "The todo item to add"},
                        "id": {"type": "integer", "description": "The id of the item to complete"}
                    },
                    "required": ["action"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "calculate",
                "description": "Perform a simple calculation",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "operation": {"type": "string", "enum": ["add", "subtract", "multiply", "divide"]},
                        "x": {"type": "number", "description": "The first number"},
                        "y": {"type": "number", "description": "The second number"}
                    },
                    "required": ["operation", "x", "y"]
                }
            }
        }
    ]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )

    response_message = response.choices[0].message

    if response_message.tool_calls:
        tool_call = response_message.tool_calls[0]
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        if function_name == "get_current_weather":
            function_response = get_current_weather(
                location=function_args.get("location"),
                unit=function_args.get("unit")
            )
        elif function_name == "convert_currency":
            function_response = convert_currency(
                amount=function_args.get("amount"),
                from_currency=function_args.get("from_currency"),
                to_currency=function_args.get("to_currency")
            )
        elif function_name == "manage_todo":
            function_response = manage_todo(
                action=function_args.get("action"),
                item=function_args.get("item"),
                id=function_args.get("id")
            )
        elif function_name == "calculate":
            function_response = calculate(
                operation=function_args.get("operation"),
                x=function_args.get("x"),
                y=function_args.get("y")
            )
        
        messages.append(response_message)
        messages.append(
            {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": function_response
            }
        )
        
        second_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        return second_response.choices[0].message.content
    else:
        return response_message.content

In [5]:
print(run_conversation("What's the weather like in Tokyo?"))
print(run_conversation("Convert 100 USD to EUR"))
print(run_conversation("Add 'Buy groceries' to my todo list"))
print(run_conversation("What's on my todo list?"))
print(run_conversation("Mark the first item on my todo list as complete"))
print(run_conversation("Calculate 15 times 7"))

The current weather in Tokyo is 22°C and sunny, with some windy conditions.
100 USD is approximately 85 EUR. Please note that exchange rates can fluctuate, so the actual amount may vary slightly.
I've added "Buy groceries" to your todo list!
Your to-do list currently has one item: **Buy groceries**.
I've marked "Buy groceries" as complete on your todo list. If you need to do anything else, just let me know!
15 times 7 is 105.
